In [5]:
#import boto3, re, sys, math, json, os, sagemaker, urllib.request
import boto3, os
import sagemaker
from sagemaker import get_execution_role
from sagemaker.sklearn.processing import SKLearnProcessor

region = boto3.session.Session().region_name

role = get_execution_role()
sklearn_processor = SKLearnProcessor(
    framework_version="0.20.0", role=role, instance_type="ml.m5.4xlarge", instance_count=1
)

In [6]:
input_data='s3://dgx-ds-use1-dev-landing-s3/kamal/input/datathon_full.zip'

In [16]:
from datetime import datetime

dt_string = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
print("date and time =", dt_string)

date and time = 2021-10-28-20-17-14


In [17]:
from sagemaker.processing import ProcessingInput, ProcessingOutput


sklearn_processor.run(
    job_name="kamal-sagemaker-filesplit-" + dt_string,
    code="utilfilesplit.py",
    inputs=[ProcessingInput(source=input_data, destination="/opt/ml/processing/input")],
    outputs=[
        ProcessingOutput(output_name="split_data", source="/opt/ml/processing/split", destination="s3://dgx-ds-use1-dev-landing-s3/kamal/input"),
    ],
    arguments=["--train-test-split-ratio", "0.2"],
)

preprocessing_job_description = sklearn_processor.jobs[-1].describe()

output_config = preprocessing_job_description["ProcessingOutputConfig"]
for output in output_config["Outputs"]:
    if output["OutputName"] == "split_data":
        preprocessed_training_data = output["S3Output"]["S3Uri"]



Job Name:  kamal-sagemaker-filesplit-2021-10-28-20-17-14
Inputs:  [{'InputName': 'input-1', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://dgx-ds-use1-dev-landing-s3/kamal/input/datathon_full.zip', 'LocalPath': '/opt/ml/processing/input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-993818375429/kamal-sagemaker-filesplit-2021-10-28-20-17-14/input/code/filesplit.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'split_data', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://dgx-ds-use1-dev-landing-s3/kamal/input', 'LocalPath': '/opt/ml/processing/split', 'S3UploadMode': 'EndOfJob'}}]
............................./miniconda3/lib/python3.7/site-packages/sklearn/externals/

In [ ]:
#onetime move file to s3
bucket_name='s3://dgx-ds-use1-dev-landing-s3'
prefix='kamal/input'

#boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, '/ordertat_data.zip')).upload_file('datathon_full.tab.zip')

s3 = boto3.client('s3')
with open('datathon_full.zip', 'rb') as data:
    s3.upload_fileobj(data, 'dgx-ds-use1-dev-landing-s3', 'kamal/input/datathon_full.zip')